In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

In [2]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"GPUs detected: {len(gpus)}")
    for gpu in gpus:
        print(f"GPU Name: {gpu}")
else:
    print("No GPU detected. Running on CPU.")

GPUs detected: 1
GPU Name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
diseases = os.listdir('../../datasets/data21k')
print(diseases)
print(f"number of diseases: {len(diseases)}")

['CBB', 'CBSD', 'CGM', 'CH', 'CMD']
number of diseases: 5


In [4]:
disease_labels = []

for disease in diseases:
   all_images = os.listdir('../../datasets/data21k/'+ disease)

   for image in all_images:
      disease_labels.append((disease, str(image)))

print(disease_labels[:5])

[('CBB', '1000015157.jpg'), ('CBB', '1001320321.jpg'), ('CBB', '1003888281.jpg'), ('CBB', '1012426959.jpg'), ('CBB', '1018973237.jpg')]


In [5]:
df = pd.DataFrame(data=disease_labels, columns=['Labels', 'Image'])
df.tail()

,Labels,Image
21392,CMD,997485103.jpg
21393,CMD,997651546.jpg
21394,CMD,997857988.jpg
21395,CMD,999068805.jpg
21396,CMD,999329392.jpg


In [6]:
print("total number of images: ", len(df))
print("total number of labels: ", len(df['Labels'].unique()))

total number of images:  21397
total number of labels:  5


In [7]:
IMAGE_SIZE = 224

path = '../../datasets/data21k/'
dataset_path = os.listdir(path)

images = []
labels = []

for i in dataset_path:
   data_path = path + str(i)
   filenames = [i for i in os.listdir(data_path)]

   for f in filenames:
      img = cv2.imread(data_path + '/' + f)
      img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
      images.append(img)
      labels.append(i)

KeyboardInterrupt: 

: 

In [8]:
images = np.array(images)
images = images.astype('float32') / 255.0
images.shape

(21397, 224, 224, 3)

In [9]:
y = df['Labels'].values
print(y)
y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

['CBB' 'CBB' 'CBB' ... 'CMD' 'CMD' 'CMD']
[0 0 0 ... 4 4 4]


In [10]:
y = y.reshape(-1, 1)
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y)
print(Y[:5])

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0


In [11]:
images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(17117, 224, 224, 3)
(17117, 5)
(4280, 224, 224, 3)
(4280, 5)


In [16]:
from tensorflow.keras import layers # type: ignore
from tensorflow.keras.applications import EfficientNetV2M # type: ignore

NUM_CLASSES = 5
IMG_SIZE = 480
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

outputs = EfficientNetV2M(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [21]:
class DataGenerator(Sequence):
   def __init__(self, x_set, y_set, batch_size):
      self.x, self.y = x_set, y_set
      self.batch_size = batch_size

   def __len__(self):
      return int(np.ceil(len(self.x) / float(self.batch_size)))

   def __getitem__(self, idx):
      batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
      batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
      return batch_x, batch_y
   
train_gen = DataGenerator(train_x, train_y, 32)
test_gen = DataGenerator(test_x, test_y, 32)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

hist = model.fit(train_gen, validation_data=test_gen, epochs=10, batch_size=32, verbose=2)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 480, 480, 3)]     0         


                                                                 
 efficientnetv2-m (Functiona  (None, 5)                53156793  
 l)                                                              
                                                                 
Total params: 53,156,793
Trainable params: 52,864,761
Non-trainable params: 292,032
_________________________________________________________________
Epoch 1/10


TypeError: in user code:

    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\compile_utils.py", line 265, in _